### Direct Get to Drive

In [ ]:
#@title { form-width: "2%" }
#@markdown #### Setup

import os, pytz, time, re
from bs4 import BeautifulSoup as Soup, element
from google.colab import drive
from datetime import datetime


tz = pytz.timezone('Turkey')
os.environ['TZ'] = 'Turkey'
nowstr = lambda separete = False: datetime.now(tz).strftime("%y-%m-%d-%H-%M-%S") if separete else str(round(time.time()))

ddyPtr = re.compile("mydaddy[^']+(?=')")
hqpVarsPtr = re.compile('(?<=;)[a-z]{8}(?==)')
fileExtPtr = re.compile('\.([a-zA-Z0-9]+)$')
hqpTitlePtn = re.compile("(?<=<title>)[\sa-zA-Z0-9]+(?=\s-)")

if not os.path.exists("/content/drive"):
    drive.mount('/content/drive')

def download(directurl, save_dir= "/content/Downloads", name= ""):
    tempsave = "/content/done/" + nowstr() 
    name = name.strip()

    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(tempsave, exist_ok=True)

    if name:
        extSch = fileExtPtr.search(directurl)
        ext = extSch.group() if extSch else ""
        filename = f' -O "{tempsave}/{name + ext}"' 
    else:
        filename = ""
    !wget -P "$tempsave"$filename "$directurl"
    !mv "$tempsave"/* "$save_dir"

def hqpGetInnerURL(url):
    reqOutput = !curl $url
    outputStr = "".join(reqOutput)
    srch = ddyPtr.search(outputStr)
    if srch:
        titleSH = hqpTitlePtn.search(outputStr)
        title = titleSH.group() if titleSH else ""
        innerURL = f"https://{srch.group()}"
        return (innerURL, title, outputStr)
    else:
        print("Search is ", type(srch))
        return (None, None, outputStr)

def hqpDCCDownload(hqpURL, saveDir= "/content", name= ""):
    print(f"Looking for dcc url in: {hqpURL}\n\t...\n")
    reqOutput = !curl $hqpURL
    invalid = False
    if reqOutput:
        if not name:
            name = nowstr(True) + "- 1080p"
        newOutputStr = reqOutput[0]
        varRes = hqpVarsPtr.findall(newOutputStr)
        if varRes:
            valDict = {}
            for mtch in varRes:
                valPt = f'(?<={mtch}=")(\/\/|\d)[^"]+'
                valSch = re.search(valPt, newOutputStr)
                if not valSch:
                    print(f"Val for {mtch} wasn't found!")
                    invalid = True
                    break
                valDict[mtch] = valSch.group()
            if not invalid:
                val1Sch = re.search('(?<=replaceAll\("nrpuv",)[a-z]{8}', newOutputStr)
                val2Sch = re.search('(?<=\+"pubs\/"\+)[a-z]{8}', newOutputStr)
                if val1Sch and val2Sch:
                    val1 = val1Sch.group()
                    val2 = val2Sch.group()
                    vidUrl = f"https:{valDict[val1]}pubs/{valDict[val2]}/1080.mp4"
                else:
                    invalid = True
            if not invalid:
                print(f"Found URL:\n\t{vidUrl}\n\n")
                download(vidUrl, save_dir= saveDir, name= name)
            else:
                print(valDict, "\n", f"val1: {val1}, val2: {val2}")


In [ ]:
#@title { form-width: "2%" }
#@markdown #### Direct

Direct_URL = "" #@param {type:"string"}
Save_Dir = "" #@param {type:"string"}
Name = "" #@param {type:"string"}

if not Name.endswith(".mp4") and not Name.endswith(".mkv"):
    Name += ".mp4"

if Direct_URL:
    download(Direct_URL, save_dir= Save_Dir, name= Name)

In [ ]:
#@title { form-width: "2%" }
#@markdown #### Find

loc = "" #@param {type:"string"}
srch = "" #@param {type:"string"}
!find "$loc" -name "*$srch*"

In [ ]:
#@title { form-width: "2%" }
#@markdown #### JUST CURL

URL = "" #@param {type:"string"}

!curl $URL

In [ ]:
#@title { form-width: "2%" }
#@markdown #### HQP

VidURLs = "" #@param {type:"string"}
Save_Dir = "" #@param {type:"string"}
Names = "" #@param {type:"string"}

urls = VidURLs.split(",,,")
names = Names.split(",,,")
if len(urls) == len(names):
    print(f"Downloading {len(urls)} links.\n")
    for i in range(len(urls)):
        url = urls[i].strip()
        name = names[i].strip().title()
        if url and name:
            hqpDCCDownload(url, saveDir= Save_Dir, name= name)
        else:
            print("URL or Name is empty!")
else:
    raise("URLs and Names have different length!")

In [ ]:
#@title { form-width: "2%" }
#@markdown
